# TAL aplicado al análisis del discurso de los medios de prensa 📰🤓🔥


El proyecto consiste en entrenar y evaluar varios modelos de clasificación supervisada capaz de clasificar una noticia según la taxonomía siguiente: 


In [1]:
topics = [
    'mundo', 'economía', 'política y conflictos', 'ciencia y tecnología',
    'catástrofes y accidentes', 'cultura y arte', 'deporte', 'ecología y planeta',
    'crimen, delitos y justicia', 'salud'
]

- Hito Unidad 1 (29 de septiembre): Datasets de entrenamiento y test + primer modelo baseline

- Hito Unidad 2 (27 de octubre): Implementación y experimentos de varios modelos de clasificación

- Hito Proyecto (15 de diciembre): Evaluación y comparación de los modelos de los distintos equipos + integración de los mejores modelos en la arquitectura Sophia2.


### índex

1. [Importación del dataset de entrenamiento](a)
2. [Extracción de términos clave de columnas title+text](b)
3. [Clasificación del dataset de entrenamiento](c)
4. [Balance de clases dataset de entrenamiento](d)
5. [Importación de los dataset restantes](e)
6. [Dividir dataset en entrenamiento y test](f)
7. [Dividir dataset entrenamiento en entrenamiento y validación](g)



In [2]:
import warnings
warnings.filterwarnings('ignore')
%matplotlib notebook

# Data manipulation
import re
import multiprocessing
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# NLP
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import Word2Vec, CoherenceModel
#import pyLDAvis
#import pyLDAvis.gensim_models  # don't skip this
import nltk 
#nltk.download('stopwords') 
import spacy
from spacy.matcher import Matcher
nlp = spacy.load("es_core_news_sm")
print("versión de Spacy: ", spacy.__version__)

versión de Spacy:  3.1.3


### 1. Importación del dataset de entrenamiento <a name="a"></a>

In [3]:
dataset = pd.read_csv("data/chile_2020-09.csv")[:100] #x mientras para facilitar el desarrollo
print("Largo de dataset: ", len(dataset))
display(dataset.head(3))

Largo de dataset:  100


,id,country,media_outlet,url,title,text,date,year,id_journalist
0,49234,chile,horas24,https://www.24horas.cl/coronavirus/tia-pikachu...,"""Tía Pikachu"" por parte empadronado: ""Estoy co...","Ha sufrido los embates del carro lanza aguas,...",2020-09-01,2020.0,NaN
1,49252,chile,horas24,https://www.24horas.cl/nacional/tia-pikachu-po...,"""Tía Pikachu"" por parte empadronado: ""Estoy co...",NaN,2020-09-01,2020.0,NaN
2,49277,chile,horas24,https://www.24horas.cl/nacional/los-toscanini-...,Los Toscanini: Cae banda que abastecía a crimi...,NaN,2020-09-01,2020.0,NaN


In [4]:
content = dataset["title"] + '. ' + dataset["text"]
content.replace(" ",np.nan,inplace=True)
content.dropna(inplace=True)
dataset = pd.DataFrame({'content':content, 'terms': ""})
display(dataset.head(3))

,content,terms
0,"""Tía Pikachu"" por parte empadronado: ""Estoy co...",
3,RM alcanza sus mejores índices de calidad de a...,
6,Paro de camioneros: Supermercados preocupados ...,


### 2.  Extracción de términos clave de columna title+text <a name="b"></a>

In [ ]:
# limpiar noticias de símbolos raros

In [6]:
matcher = Matcher(nlp.vocab)

# Pattern 1: NOUN de NOUN
pattern_1 = [{"POS": "NOUN"},{"LOWER": "de"}, {"POS": "NOUN"}]
matcher.add("NOUN-de-NOUN", [pattern_1])

# Pattern 2: NOUN ADJ
pattern_2 = [{"POS": "NOUN"}, {"POS": "ADJ"}]
matcher.add("NOUN-ADJ", [pattern_2])

# Pattern 3: 
pattern_3 = [{"POS": "NOUN"}]
matcher.add("NOUN", [pattern_3])



    Bloque de alto coste computacional ⬇️

In [7]:
for index, row in dataset.iterrows():
    doc = nlp(row['content'])
    matches = matcher(doc)
    categories = ""
    for match_id, start, end in matches:
        string_id = nlp.vocab.strings[match_id]  # Get string representation
        span = doc[start:end]  # The matched span
        categories = categories + span.text + "; "
    
    row['terms'] = categories
    
display(dataset.head(3))

,content,terms
0,"""Tía Pikachu"" por parte empadronado: ""Estoy co...",parte; parte empadronado; diciembre; embates; ...
3,RM alcanza sus mejores índices de calidad de a...,índices; índices de calidad; calidad; registro...
6,Paro de camioneros: Supermercados preocupados ...,Paro; Paro de camioneros; camioneros; desabast...


In [9]:
from collections import Counter
dataset["most_common"] = ""
for index, row in dataset.iterrows():
    split_it = row['terms'].split("; ")
    counter = Counter(split_it)
    row['most_common'] = counter.most_common()[0][0]

display(dataset.head(30),dataset['most_common'].value_counts())

,content,terms,most_common
0,"""Tía Pikachu"" por parte empadronado: ""Estoy co...",parte; parte empadronado; diciembre; embates; ...,parte
3,RM alcanza sus mejores índices de calidad de a...,índices; índices de calidad; calidad; registro...,aire
6,Paro de camioneros: Supermercados preocupados ...,Paro; Paro de camioneros; camioneros; desabast...,país
7,Sernac recibió más de 400 reclamos durante la ...,reclamos; jornada; jornada de CyberDay; CyberD...,reclamos
8,Denuncian desabastecimiento en el sur del país...,desabastecimiento; sur; país; paro; paro de ca...,desabastecimiento
9,PDI encuentra a hombre que familia reportó des...,hombre; años; hallazgo; vida; hombre; familiar...,hombre
10,"Bellolio: ""No hay ninguna razón para que prosp...",razón; acusación; acusación constitucional; mi...,ministro
13,"Ministro Pérez pide ""actuar con responsabilida...",responsabilidad; ultimátum; ultimátum de acusa...,responsabilidad
16,Alcaldes piden que transporte sea gratuito el ...,transporte; día; plebiscito; Alcaldes; comunas...,transporte
17,Paro de camiones: DC da ultimátum para que min...,Paro; Paro de camiones; camiones; ultimátum; m...,ministro


camioneros    4
ministro      4
país          3
querella      3
casos         3
             ..
cafeterías    1
renuncia      1
aire          1
pena          1
hueones       1
Name: most_common, Length: 72, dtype: int64

### 3. Clasificación del dataset de entrenamiento <a name="c"></a>
Aplicaremos LDA a cada noticia y extraeremos sus tópicos, luego evaluaremos esos tópicos con las categorias deseadas.


In [10]:
dataset['category'] = ""
#for index, row in dataset.iterrows():
#    for index in topics:

dataset

,content,terms,most_common,category
0,"""Tía Pikachu"" por parte empadronado: ""Estoy co...",parte; parte empadronado; diciembre; embates; ...,parte,
3,RM alcanza sus mejores índices de calidad de a...,índices; índices de calidad; calidad; registro...,aire,
6,Paro de camioneros: Supermercados preocupados ...,Paro; Paro de camioneros; camioneros; desabast...,país,
7,Sernac recibió más de 400 reclamos durante la ...,reclamos; jornada; jornada de CyberDay; CyberD...,reclamos,
8,Denuncian desabastecimiento en el sur del país...,desabastecimiento; sur; país; paro; paro de ca...,desabastecimiento,
...,...,...,...,...
95,"Rodolfo Carter: ""Ser comunista hoy es ser mili...",militante; comparación; alcalde; militante; mi...,país,
96,Municipio afirma que baja de ventas llevó a ce...,ventas; baja; ventas; cierre; casos; zona; con...,ventas,
97,Corte declara admisible recurso de protección ...,recurso; recurso de protección; protección; re...,protección,
98,"""Matamos a cuanto hueón pillamos"": audio inédi...",cuanto; hueón; audio; audio inédito; confesion...,hueones,


### 4.  Balance de clases dataset de entrenamiento <a name="d"></a>
Visualizamos la densidad de los tópicos con tal de combatir el sobreajuste si corresponde

In [ ]:
fig, ax = plt.subplots(1, figsize=(9,5), tight_layout=True)
ax.hist(dataset['category'],edgecolor = 'black',bins=dataset['category'].unique())
plt.xticks(rotation='vertical', fontsize=7)
plt.ylabel("densidad")
plt.show()